In [ ]:
# pystac client helps search the stac api
! pip install pystac-client

In [ ]:
from pystac_client import Client
import rasterio as rio
from rasterio.windows import Window
from shapely.geometry import box, shape
import geopandas as gpd
from rasterio.windows import from_bounds
import requests
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO

In [ ]:
CEDA_STAC_API = "https://api.stac.ceda.ac.uk/"

In [ ]:
S2_ARD_COLLECTION = "sentinel2_ard"


In [ ]:
catalog = Client.open(CEDA_STAC_API)

In [ ]:
bbox = [-2.79020226,55.95618749,-2.76890687,55.96888997]

In [ ]:
start_date = "2025-05-01"

end_date = "2025-05-31"

daterange = f"{start_date}/{end_date}"

In [ ]:
# see what other properties can query for this collection
collection = catalog.get_collection(S2_ARD_COLLECTION)
collection.get_queryables()

In [ ]:
# will use a max cloud_cover
cloud_filter = {
    "op": "<=",
    "args": [{"property": "cloud_cover"}, 10]
}

In [ ]:
# Perform the search
search = catalog.search(
    collections=[S2_ARD_COLLECTION],
    bbox=bbox,
    datetime=daterange,
    filter={"op": "and", "args": [cloud_filter]},
)

search_items = search.item_collection()

print(f"{len(search_items)} items returned")

# You can then iterate through the found items
for item in search_items:
    print(item.id)

In [ ]:
# check a given image intersects the aoi
first_item = search_items[0]

bbox_polygon = box(*bbox)
image_footprint = shape(first_item.geometry)

intersection = bbox_polygon.intersection(image_footprint)

coverage_proportion = intersection.area / bbox_polygon.area

print(f"Image footprint covers {coverage_proportion:.2%} of your input bbox.")

In [ ]:
# assets of items can be downloaded or read
first_item.assets.keys()

In [ ]:
# Download the cog
download_url = first_item.assets["cog"].href
file_name = download_url.split("/")[-1]

In [ ]:
# Download the whole tile - commenting this out as takes a while, large download

# with requests.get(download_url, stream=True) as r:
#     r.raise_for_status()
#     with open(file_name, 'wb') as f:
#         for chunk in r.iter_content(chunk_size=8192):
#             f.write(chunk)

# print(f"Successfully downloaded to '{file_name}'")

In [ ]:
# read / write an array of cog from bbox
bbox_polygon = box(*bbox)
box_gdf = gpd.GeoDataFrame(geometry=[bbox_polygon], crs="EPSG:4326").to_crs("epsg:27700")
bounds = box_gdf.total_bounds

output_filename = "clipped_test.tif"

with rio.open(download_url) as src:

    window = from_bounds(*bounds, src.transform)

    # Read bands 1, 2, and 3 from the window
    data = src.read([1, 2, 3], window=window)

    out_transform = src.window_transform(window)

    profile = src.profile.copy()
    profile.update({
        'height': int(window.height),
        'width': int(window.width),
        'transform': out_transform,
        'count': 3
    })

    # Write the new GeoTIFF to a local file
    with rio.open(output_filename, 'w', **profile) as dst:
        dst.write(data)
        
    print(f"Successfully saved {output_filename}")

In [ ]:
# Use the thumbnail asset to preview the image, e.g. before download

thumbnail_asset = first_item.assets['thumbnail']
thumbnail_url = thumbnail_asset.href
print(f"Thumbnail URL: {thumbnail_url}")

print("Thumbnail asset not found in this item.")
exit()
response = requests.get(thumbnail_url)
response.raise_for_status() # This will raise an error for bad responses (4xx or 5xx)

# 3. Open the image from the downloaded content
# BytesIO allows PIL to read the image data directly from memory
thumbnail_image = Image.open(BytesIO(response.content))

# 4. Plot the image using Matplotlib
print("Displaying thumbnail...")
plt.figure(figsize=(8, 8)) # You can adjust the figure size
plt.imshow(thumbnail_image)
plt.title(f"Thumbnail for Item: {first_item.id}")
plt.axis('off') # Hide the axes for a cleaner look
plt.show()